# Deep dive decision trees

Notebook bij de les over de werking van decision trees
© Auteur: Rianne van Os (ism gemini 2.5)


**Voorbereiding**
Het is belangrijk dat je al eens recursief een boomstructuur opgebouwd hebt met een Tree en een Node klasse. Dit kun je oefenen in het notebook recursie_les1.ipynb

In de machine learning lessen heb je geleerd wat een Decision Tree is en hoe je die moet trainen met behulp van sklearn. In deze les gaan we dit algoritme beter bekijken en een deel ervan zelf implementeren.

*Note*: zet je copilot uit en gebruik geen generatieve AI voor deze opdrachten. Je gaat aan de hand van kleine stappen een eigen beslisboom maken, uiteraard kan iedere LLM deze stappen zo voor je uitwerken. Daar leer je niks van en je ontneemt jezelf ook het plezier van het in staat zijn een werkend algoritme te implementeren.  

In [ ]:
#importeer benodigde libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Decision trees - korte herhaling
We hebben al eerder een decision tree getrained en deze gevisualiseerd. Hieronder zie je een stukje van een boom die we in de eerste les over machine learning getraind hebben:

![dt](./../afbeeldingen/WiskundigeTechnieken/decisiontree.png "Stuk van decision tree")

We zien hier per node van de boom een aantal zaken:
- Op welke feature er gesplitst wordt en op welke splittingswaarde (threshold) dat gebeurt
- De gini impurity index van de Node. Dit geeft aan hoe zuiver of onzuiver de node is.
- Hoeveel samples er in totaal in de node zitten.
- Hoeveel samples er van iedere soort in die node zitten (dit is de parameter value)
- De soort die het meest voorkomt in die node. Dit zou de voorspelling zijn.

De laatste nodes, die we *leaf nodes* noemen, zijn hier helemaal zuiver: er komen alleen nog pinguins van 1 soort voor (en deze boom is dus waarschijnlijk overfit).

Om zo'n boom te maken moet voor iedere nieuwe node bepaald worden op welke feature gesplitst moet worden en op welke threshold dat moet gebeuren. Vervolgens kan dan zo'n boom recursief opgenbouwd worden. Hoe dit precies gebeurt ga je in de volgende opdrachten zien en uiteindelijk ga je dit allemaal zelf implementeren.



## Het bepalen van de split
Er zijn twee maten waarmee bepaald kan worden wat voor een gegeven node de beste volgende split gaat zijn. Dit kan op basis van de *gini impurity index* of op basis van *entropie*. In de boom hierboven zag je dat er gebruik gemaakt werd van de gini index, maar wij gaan kijken naar entropie. 

Om uit te leggen wat entropie is, gebruiken we onderstaand datasetje. Daarin staan gegevens over studenten die wel of niet een vak gehaald hebben. We willen kijken of je op basis van de vooropleiding en het aantal uur dat de student geleerd heeft kunt voorspellen of het vak gehaald wordt.

In [ ]:
resultaten_df = pd.DataFrame({'vooropleiding': ['MBO', 'MBO', 'anders', 'havo', 'anders', 'MBO', 'havo'],
                     'uur': [23, 18, 32, 28, 11, 15, 2], 
                     'gehaald': [True, True, True, False, False, False, False]})
resultaten_df

We kunnen de eerste split maken op de kolom vooropleiding of op de kolom uur. Wat zou logisch zijn? En welke splittingswaarde moeten we gebruiken? Stel we kiezen vooropleiding als kolom om eerst op te splitten, gaan we dan voor `vooropleiding == MBO` en `vooropleiding != MBO`, of zou `havo` een betere eerste keuze zijn? En kiezen we voor de kolom uur, gaan we dan voor `uur <16.5`? Of zou een andere waarde beter zijn?

### Opdracht 1
Bedenk zelf op welke feature je als eerst gaat splitten en wat de bijbehorende splittingswaarde is. Leg uit waarom dat een logische keuze is.

## Entropie

Waarschijnlijk heb je bij bovenstaande opdracht gekeken hoe je een split kunt kiezen die ervoor zorgt dat te `true` en `false` waarden zo goed mogelijk gescheiden worden in 2 nodes. We zeggen hier dat je kiest voor een split die de nodes zo *zuiver* mogelijk maken. Waarbij een *zuivere* node enkel datapunten heeft uit 1 enkele klasse, en een *onzuivere* node heeft datapunten uit verschillende klassen. Nu helpt het om die 'zuiverheid' uit te kunnen berekenen. De maat die daarvoor gebruikt kan worden is *entropie*. (Een andere maat is gini-impurity, die mag je zelf uitzoeken voor boven niveau)

Entropie kun je als volgt berekenen: voor een set data `S` met verschillende klassen, wordt de Entropie `H(S)` berekend als:

$ H(S) = - \sum_{i=1}^{c} p_i \log_2(p_i) $

Hierbij geldt:
*   $ c $ is het aantal unieke klassen.
*   $ p_i $ is het aandeel van samples in set `S` die behoren tot klasse $ i $. Dus bijvoorbeeld 2/7 als 2 van de 7 datapunten bij klasse $i$ horen.

Merk verder op dat we het logaritme nemen met grondtal 2. In numpy is dit `np.log2(x)`.


Bestaat de set S van labels maar uit twee klassen (bijvoorbeeld True en False), dan kunnen de we formule makkelijk zonder sommatie-teken schrijven en dan krijg je:
    $$ H(S) = - p \log_2(p) - (1-p) \log_2(1-p) $$
Waarbij $p$ het aandeel True-labels is (dus opnieuw bijvoorbeeld 2/7 als 2 van de 7 labels True is).

### Opdracht 2
Beantwoord de volgende vragen door de berekeningen handmatig (met pen en papier) uit te voeren:

1. Bereken de entropie van bovenstaande datasetje, als we uitgaan van de kolom `gehaald`.
2. Je kunt deze berekening voor entropie ook gebruiken als je meer dan 2 klasses hebt. Stel dat we een decision tree hebben getraind om op basis van gegeven features te voorspellen of een datapunt bij klasse A, B of C hoort. Een bepaalde node in de decision tree bestaat uit 10 samples, waarvan 5 uit klasse A, 3 uit klasse B en 2 uit klasse C. Wat is de entropie van deze node?
3. Wat is de entropie van een node die bestaat uit 3 datapunten van klasse A en 3 uit klasse B?
4. Wat is de entropie van een node die bestaat uit 4 datapunten van klasse B?

In [ ]:
#uitwerking 1
-3/7*np.log2(3/7) - 4/7 * np.log2(4/7)

In [ ]:
#uitwerking 2
-5/10*np.log2(5/10) - 3/10 * np.log2(3/10)-2/10*np.log2(2/10)

In [ ]:
#uitwerking 3
-3/6*np.log2(3/6) - 3/6 * np.log2(3/6)

In [ ]:
#uitwerking 4
-1*np.log2(1)

### Entropie bij 2 klassen
Als er twee klassen zijn (dus $c = 2$), dan ligt de waarde van de entropie tussen de 0 (alles behoort tot één klasse) en 1 (meetwaarden zijn precies verdeeld over de twee klassen).

NB. Als je het aandeel $p_1$ van de eerste klasse $c_1$ weet, dan weet je ook het aandeel $p_2 = 1 - p_1$ van de tweede klasse $c_2$. Bij méér dan twee klassen gaat dit natuurlijk niet meer op. 

Bij 2 klassen geeft de volgende grafiek de verandering in entropie weer ten opzichte van het aandeel van klasse $c_1$.

In [ ]:
fig, ax = plt.subplots(1, dpi=100)

x = np.linspace(.001,.999,100)
y = -x * np.log2(x) + -(1-x) * np.log2(1-x)
ax.set_xlabel(r"Aandeel van klasse $c_1$")
ax.set_ylabel(r"Entropie $E(S)$")

plt.plot(x,y)
plt.show()

## Opdracht 3
1. Schrijf een functie `calculate_entropy` die de entropie van een numpy array met labels kan berekenen. Deze moet ook werken voor een classificatieprobleem met meer dan 2 klassen. Controleer hiermee je antwoorden uit de vorige opdracht.
2. Bereken met deze functie de entropie van de hele dataset (dus van de kolom `gehaald`). Bereken daarna de entropie van de nodes die je krijgt als je resultaten_df splitst op `vooropleiding == MBO`. Doe hetzelfde voor de split `uur < 16.5`. Kun je op basis hiervan al bedenken wat een betere split zou zijn?

In [ ]:
def calculate_entropy(labels = np.array) -> float:
    pass

In [ ]:
print(f'Entropie van hele kolom {calculate_entropy(resultaten_df['gehaald'])}')

In [ ]:
print(f"Entropie van vooropleiding = MBO: {calculate_entropy(resultaten_df.loc[resultaten_df['vooropleiding']=='MBO','gehaald'])}")
print(f"Entropie van vooropleiding != MBO: {calculate_entropy(resultaten_df.loc[resultaten_df['vooropleiding']!='MBO','gehaald'])}")

In [ ]:
print(f"Entropie van uur <=15: {calculate_entropy(resultaten_df.loc[resultaten_df['uur']<=16.5,'gehaald'])}")
print(f"Entropie van uur > 15 {calculate_entropy(resultaten_df.loc[resultaten_df['uur']>16.5,'gehaald'])}")

## Information gain

Met de entropie kunnen we uitrekenen hoe zuiver 1 node in de boom is, maar daarmee hebben we nog geen goede split bepaald. Daarvoor hebebn we een nieuwe term nodig, namelijk de *information gain*. Om de information gain uit te rekenen, bepaal je het verschil tussen de entropie van de *parent node* en de *gewogen gemiddelde* entropie van de *child nodes*. We willen namelijk dat de nieuwe nodes (*child nodes*) zuiverder zijn dan de oorspronkelijke node (*parent node*). Een hogere Information Gain betekent dan een betere split. Het is de split die de meeste "informatie" oplevert of de "onzekerheid" het meest reduceert. 

De formule voor de information gain is:

$ IG(P, Children) = H(P) - ( \frac{|C_1|}{|P|} H(C_1) + \frac{|C_2|}{|P|} H(C_2) )$

*   $ H(P) $ is de entropie van de *parent node*.
*   $ |C_i| $ is het aantal samples in *child node* $ C_i $.
*   $ |P| $ is het aantal samples in de *parent node*.
*   $ H(C_i) $ is de entropie van kind-set $ C_i $.
*   $ \frac{|C_i|}{|P|} $ is het gewicht van de *childe node*, gebaseerd op het aantal samples.

### Opdracht 4
Bereken de infomation gain voor de split van `resultaten_df` op `vooropleiding == MBO` en die op `uur < 16.5`. Welke heeft de hoogste *information gain* en is dus de beste split?

### Opdracht 5
Implementeer een functie `calculate_information_gain`. De eerste parameter is een numpy array met labels uit de *parent node* en de tweede parameter is een lijst met 2 numpy arrays met de labels van de 2 *child nodes*. Maak bij de implementatie gebruik van de functie `calculate_entropy`. Controleer hiermee de gevonden waarde uit de vorige opdracht.

In [ ]:
def calculate_information_gain(parent_labels : np.array, child_labels : list) -> float:
    pass

## Data splitten

Nu we weten hoe je de kwaliteit van een split kunt bepalen, kunnen we toe gaan werken naar een functie die, gegeven een dataset, zelf de beste split kan bepalen. Voordat we die functie implementeren, starten we eerst met een functie split_data, die gegeven de features, de target, de feature waarop gesplit moet worden (bijvoorbeeld `vooropleiding`) en de splittingswaarde (bijvoorbeeld `mbo`), de labels van de child-nodes teruggeeft. Deze functie kun je vervolgens gebruiken om daadwerkelijk de beste split te gaan bepalen.

**Merk op:** Het splitten van een categorische variabele werkt net iets anders dan bij numerieke variabele. Deze functie heeft daarom ook nog een paramater `is_numeric` nodig om te bepalen hoe gesplit moet worden.

### Opdracht 6
Implementeer de functie `split_data`.

In [ ]:
def split_data(features: pd.DataFrame, target: np.array, feature: str, threshold: any, is_numeric: bool = False) -> tuple:
    pass

In [ ]:
def split_data(features: pd.DataFrame, target: np.array, feature: str, threshold: any, is_numeric: bool = False) -> list:
    """
    Split the data based on a feature and a threshold.
    
    Args:
        features (pd.DataFrame): DataFrame with features
        target (pd.Series): Series with target labels
        feature (str): Feature to split on
        threshold (str): Threshold value for the split
        is_numeric (bool): Whether the feature is numeric or categorical
        
    Returns:
        list: List with two arrays containing the labels of the child nodes
    """
    pass

#test
split_data(resultaten_df, resultaten_df['gehaald'], 'uur', 15, is_numeric=True)

### Opdracht 7
Nu hebben we alle functies die nodig zijn om, gegeven een dataset, de beste split te bepalen. Maak gebruik van de eerder geimplementeerde functies om de functie `find_best_split` te implementeren. Deze functie krijgt de features en de target mee en returned de naam van de feature waarop gesplitst moet worden en het splitcriteruim (de threshold). We gaan er in deze implementatie vanuit dat er bij categorische waarden op iedere waarde gesplit kan worden. 

Om de beste split op een numerieke waarde te bepalen, bepaal je eerst alle gemiddelden van twee opeenvolgende waarden, en dat zijn dan je mogelijke splittingswaarden. Bijvoorbeeld, als de waarden in een kolom 1, 3, 4 en 7 zijn, dan zijn de mogelijke splittingswaarden 2 (gemiddelde van 1 en 3), 3.5 (gemiddelde van 3 en 4) en 5.5 (gemiddelde van 4 en 7). Code om dit efficient te doen zullen we je geven:
```python
    unique_values = np.unique(features[feature])
    unique_values.sort()
    thresholds = (unique_values[:-1] + unique_values[1:]) / 2
```
Bij een categorische feature zijn de splittingswaarden gewoon alle unieke categorien.

Roep de functie aan om de beste split van `resultaten_df` te bepalen. Lees vervolgens een dataset in waarop we eerder een classificatie hebben gedaan (bijvoorbeeld `pinguins.csv`) en test je functie daarop. 


In [ ]:
def find_best_split(features: pd.DataFrame, target: pd.Series) -> tuple:
    """
    Find the best split for the given features and target.
    
    Args:
        features (pd.DataFrame): DataFrame with features
        target (pd.Series): Series with target labels
        
    Returns:
        tuple: Feature name and threshold value for the best split
    """
    pass

#test
find_best_split(resultaten_df.drop(columns='gehaald'), resultaten_df['gehaald'])

In [ ]:
#test met pinguins dataset
pinguins_df = pd.read_csv('../databronnen/pinguins.csv')
pinguins_df.dropna(inplace=True)
find_best_split(pinguins_df.drop(columns='species'), pinguins_df['species'])

## Een decision tree implementeren - portfolio item

Nu zijn we zo ver dat je deze logica samen kunt gaan voegen in een DecisionTree klasse. We gaan deze beslisboom weer recursief opbouwen, net zoals je in de recursie les gedaan hebt. Ook hier ga je een Node klasse maken, die zelf bijhoudt welke labels er in die node zitten, op welke feature en op welke threshold gesplitst moet worden en wat zijn 'child nodes' zijn. 

Daarnaast implementeer je een DecisionTree class. Hierin komt alle logica bij elkaar. Deze heeft weer 2 recursieve elementen. 
- Het bouwen van de Tree, dat gedaan wordt met `Tree.fit()` waarin `Tree._build_recursive()` aangeroepen wordt.
- Het doen van een voorspelling met `Tree.predict()`. Deze kan een dataframe met meerdere rijen meekrijgen, en roept vervolgens per rij `Tree._predict_one()`, die op zijn beurt weer recursief opgebouwd wordt door `Tree._predict_one_recursive()` aan te roepen.

De base case is telkens het geval waarbij de Node een leaf is (dus waarbij Node.is_leaf = True), in alle andere gevallen moet iedere node de recursieve functie op de left en de right child aanroepen. 

Het is erg handig om de boom te kunnen printen, de code om dat recursief te doen krijg je van ons. Wel moet je `Node.__repr__()` zelf schrijven, zodat de nodes geprint kunnen worden.

Hieronder staan de klassediagrammen die tot een goede implementatie kunnen leiden. Je mag hiervan afwijken.

In [ ]:
#uitwerking voor het printen van de DT, rest van de functies moeten zelf geschreven worden.
class DecisionTree():
    
    #TODO: implementeer andere functies zelf
    
    def __repr__(self):
        # Return the full tree representation as a string so print(tree) works
        if self.root is None:
            return "<empty tree>"
        return self._repr_recursive(self.root, prefix = "")
                
    def _repr_recursive(self, node: Node, prefix="", type = None) -> str:
        """Return a string representation of the subtree rooted at `node`.

        This function builds lines recursively and returns a single string
        (with embedded newlines). 
        """
        if node is None:
            return ""

        # determine line prefix and new prefix for children
        if type == 'left':
            line_prefix = prefix + "├── L: "
            new_prefix = prefix + "│   "
        elif type == 'right':
            line_prefix = prefix + "└── R: "
            new_prefix = prefix + "    "
        else:  # root
            line_prefix = ""
            new_prefix = prefix

        lines = [line_prefix + str(node)]

        # recursively collect left and right subtree strings
        left_str = self._repr_recursive(node.left_child, new_prefix, type='left')
        right_str = self._repr_recursive(node.right_child, new_prefix, type='right')

        if left_str:
            lines.append(left_str)
        if right_str:
            lines.append(right_str)

        return "\n".join(lines)


Het klassediagram. Hiermee kun je het werkend krijgen, maar je mag ook voor andere functies kiezen. Merk op: je hebt heel veel al een keer geimplementeerd in de voorgaande opdrachten.

```mermaid
classDiagram
direction TB

class Node {
    +pd.Series target
    +str split_feature
    +any split_threshold
    +Node left_child
    +Node right_child
    +bool is_leaf
    +_get_majority_class() (unique, counts, majority)
    +__repr__() str
}

class DecisionTree {
    +Node root
    +int max_depth
    +__repr__() str
    +_calculate_entropy(np.array) float
    +_calculate_information_gain(np.array, pd.Series, pd.Series) float
    +_split_data(pd.DataFrame, np.array, str, any, bool) tuple
    +_find_best_split(pd.DataFrame, pd.Series) tuple
    +_build_recursive(pd.DataFrame, pd.Series, int) Node
    +_repr_recursive(Node, str, str) str
    +_predict_one(pd.Series)
    +_predict_one_recursive(Node, pd.Series)
    +fit(pd.DataFrame, pd.Series)
    +predict(pd.DataFrame) np.array
}


```

Zorg dat onderstaande code werkt:

In [ ]:
tree = DecisionTree(max_depth=4)
tree.fit(pinguins_df.drop(columns='species'), pinguins_df['species'])

In [ ]:
tree.predict(pinguins_df.drop(columns='species'))

In [ ]:
print(tree)

Test je decision tree ook op een andere dataset, die mag je zelf kiezen. Vergelijk de resultaten met de sklearn implemenatie van een decision tree.

Zorg ook dat je laat zien dat de max_depth parameter werkt.

## Wat lever je in?
- Een .py bestand met de `Node` en `DecisionTree` klasse
- Een notebook waarin je laat zien dat je DecisionTree werkt. Let op, hierin staat niet alleen code maar ook markdown die vertelt wat je doet en wat het resultaat is.
- De dataset die nodig is om je notebook te runnen.